# Utilizando OpenStreetMap

Vamos a utilizar ahora OpenRoute service para mostrar un caso de uso real

In [50]:
#Importar librerias

import random
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import cdist
from ortools.linear_solver import pywraplp
import time
import numpy as np
import folium
from folium.plugins import BeautifyIcon
import openrouteservice as ors
import osmnx as ox



In [51]:
# issue https://github.com/microsoft/vscode/issues/266193

from folium import Map
import base64
from IPython.display import IFrame, display

def show_folium_safe(m : Map, height=1000):
    """
    Displays a Folium map in a safe IFrame using Base64 encoding.
    This avoids "Trusted" errors, file path issues, and CSS leakage.
    """
    # 1. Get the raw HTML string of the map
    html_content = m.get_root().render()
    
    # 2. Encode the HTML to base64
    # This allows us to put the entire map "inside" the URL string
    encoded = base64.b64encode(html_content.encode('utf-8')).decode('utf-8')
    
    # 3. Create a Data URI
    data_uri = f"data:text/html;charset=utf-8;base64,{encoded}"
    
    # 4. Display the IFrame
    # We use width='100%' to fill the cell width, but the CSS is trapped inside
    display(IFrame(src=data_uri, width="100%", height=height))

In [52]:
# Mapa alrededor de Rosario

# Coordenadas del Monumento a la Bandera (aprox)
lat_monumento = -32.9478
lon_monumento = -60.6305

m = folium.Map(location=[lat_monumento, lon_monumento], zoom_start=15)

generar puntos aleatorios en rosario

In [53]:
print("Descargando red vial del Centro de Rosario...")

# Descargar calles a 1500 metros a la redonda del punto
G = ox.graph_from_point((lat_monumento, lon_monumento), dist=1500, network_type='drive')


# 2. Convertir el grafo a GeoDataFrames (nodos y aristas)
nodes, edges = ox.graph_to_gdfs(G)

# 3. Muestreo Aleatorio
# Seleccionamos 100 nodos (intersecciones) al azar de la red vial
# Esto asegura que los puntos existan físicamente en una calle
puntos_reales = nodes.sample(n=20, random_state=42)

# 4. Crear el DataFrame limpio para tu VRP
df_vrp = pd.DataFrame({
    'osmid': puntos_reales.index, # ID oficial de OpenStreetMap (útil para calcular distancias)
    'latitud': puntos_reales.y.values,
    'longitud': puntos_reales.x.values
})

print(f"Se generaron {len(df_vrp)} puntos válidos sobre calles.")
print(df_vrp.head())


Descargando red vial del Centro de Rosario...
Se generaron 20 puntos válidos sobre calles.
        osmid    latitud   longitud
0   257491698 -32.945130 -60.637867
1   257489792 -32.938708 -60.640588
2   257489020 -32.955139 -60.646171
3  3364478903 -32.952454 -60.627765
4   257490114 -32.943646 -60.640368


In [54]:
for idx, row in df_vrp.iterrows():
    folium.Marker(
        location = [row['latitud'], row['longitud']],
        icon=BeautifyIcon(
            icon_shape='marker',
            number=idx,
            border_color='blue',
            text_color='white',
            background_color='blue'
        )
    ).add_to(m)

In [55]:
show_folium_safe(m)

In [56]:
import networkx as nx
import osmnx as ox
import pandas as pd
import numpy as np

G = ox.add_edge_speeds(G) 
G = ox.add_edge_travel_times(G)

In [57]:
# Lista de los IDs de los nodos que vamos a usar
nodos_interes = df_vrp['osmid'].tolist()
n = len(nodos_interes)

# Inicializamos matrices vacías con ceros
matriz_distancia = np.zeros((n, n))
matriz_tiempo = np.zeros((n, n))

print(f"Calculando rutas entre {n} puntos... (esto puede tomar un momento)")

# Iteramos sobre cada origen y destino
for i, origen in enumerate(nodos_interes):
    for j, destino in enumerate(nodos_interes):
        if i == j:
            continue # Distancia a sí mismo es 0
        
        try:
            # Calcular distancia en metros (weight='length')
            dist = nx.shortest_path_length(G, source=origen, target=destino, weight='length')
            matriz_distancia[i][j] = dist
            
            # Calcular tiempo en segundos (weight='travel_time')
            tiempo = nx.shortest_path_length(G, source=origen, target=destino, weight='travel_time')
            matriz_tiempo[i][j] = tiempo
            
        except nx.NetworkXNoPath:
            # Si no hay camino (ej: calle cortada o sentido único imposible), ponemos un valor muy alto
            matriz_distancia[i][j] = 999999
            matriz_tiempo[i][j] = 999999

print("Cálculo terminado.")

Calculando rutas entre 20 puntos... (esto puede tomar un momento)
Cálculo terminado.


In [58]:
ids = df_vrp['osmid'].values

# dfs de distancias y tiempos
df_matrix_dist = pd.DataFrame(matriz_distancia, index=ids, columns=ids)
df_matrix_time = pd.DataFrame(matriz_tiempo, index=ids, columns=ids)

print(df_matrix_dist.iloc[:5, :5])
print(df_matrix_time.iloc[:5, :5])

             257491698    257489792    257489020    3364478903   257490114 
257491698      0.000000  1039.159428  1780.353610  1912.300513   639.883679
257489792   1030.066008     0.000000  2292.284526  2444.739573   665.809449
257489020   1986.606401  2028.515489     0.000000  2242.456727  1629.239740
3364478903  1585.061390  2386.660940  2277.549876     0.000000  1991.587555
257490114    364.256559   887.023520  1629.551591  2145.402615     0.000000
            257491698   257489792   257489020   3364478903  257490114 
257491698     0.000000  100.908349  172.287340  185.435274   65.105353
257489792    95.758976    0.000000  206.305607  178.581317   59.922850
257489020   190.986132  191.084824    0.000000  180.256413  155.150006
3364478903  144.769905  175.022581  204.979489    0.000000  156.326987
257490114    35.836125   83.091077  154.031031  179.578271    0.000000


In [59]:
df_matrix_dist

,257491698,257489792,257489020,3364478903,257490114,257491537,257490219,257490605,257490911,257490685,282227014,257489978,1650474841,257490112,257490752,1310764161,257488992,282227009,1962759197,257490364
257491698,0.000000,1039.159428,1780.353610,1912.300513,639.883679,2834.940086,1115.974646,2013.032716,1926.982715,724.299325,2193.754644,1771.011566,1551.961489,901.657499,1647.244834,3088.164271,1308.392317,2455.630183,1805.381454,1386.393872
257489792,1030.066008,0.000000,2292.284526,2444.739573,665.809449,3367.379147,1630.392853,2863.275182,2461.124004,1754.365333,3015.023975,2801.077574,1703.021260,400.560031,2477.679149,3271.497180,269.232889,2988.069244,1988.714363,2416.459880
257489020,1986.606401,2028.515489,0.000000,2242.456727,1629.239740,2126.582594,917.201574,1296.940524,1716.726714,1603.094151,1477.697506,1054.919374,2684.081254,1896.193537,1435.600589,2624.486869,2297.748378,1744.047435,2711.754751,671.222203
3364478903,1585.061390,2386.660940,2277.549876,0.000000,1991.587555,1464.256437,1637.592839,1751.656156,558.001294,946.050960,1342.199961,2254.615739,441.624526,2249.080196,841.890573,1203.457150,2655.893829,1084.946533,469.298024,1853.442097
257490114,364.256559,887.023520,1629.551591,2145.402615,0.000000,3068.042189,964.583404,2341.552500,2161.787046,1088.555884,2493.301293,2135.268125,1435.957439,754.701568,1955.956467,2972.160222,1156.256409,2688.732285,1689.377405,1750.650431
257491537,2363.274435,3164.873984,2123.123514,778.213044,2769.800599,0.000000,2000.659359,1586.775456,920.947095,1724.264004,1153.413424,2089.735039,1219.837571,3027.293241,1204.836375,1266.888345,3434.106873,890.925788,1247.511068,1688.561398
257490219,1069.404828,1622.967085,1195.262569,1327.209472,1223.691336,2249.849045,0.000000,1427.941675,1341.891674,687.846896,1608.663603,1185.920525,1768.833998,1490.645133,1062.153793,2530.666622,1892.199974,1870.539142,1796.507495,801.302831
257490605,2236.369083,3263.961082,1836.039505,1735.597876,2867.352635,1613.951055,1713.575350,0.000000,1209.163765,1585.704785,685.928995,1324.656527,2177.222402,3126.380338,928.037640,1832.718357,3533.193971,952.278924,2204.895900,923.482885
257490911,1582.199627,2609.791626,1719.548582,526.434110,2213.183178,1449.073684,1083.387761,1193.654862,0.000000,935.237590,821.234105,1696.614445,968.058637,2472.210882,283.889279,1729.891260,2879.024515,1069.763781,995.732134,1295.440803
257490685,684.141669,1711.733667,1847.406743,1190.375405,1315.125220,2113.014979,1203.968203,1321.513023,1206.759836,0.000000,1480.862119,1824.472606,1215.743778,1574.152924,943.517294,2393.832555,1980.966556,1733.705075,1243.417276,1423.298964


Ahora ya tenemos las matrices para trabajar el TSP

In [60]:
def points_sub(points, i):
    """
    De una lista de numeros sustrae el numero i en una copia nueva.
    Se considera que poinla lista no tiene elementos duplicados (por ser lista de indices) y que i se encuentra en la lista.

    Args:
      points: lista de puntos.
      i: elemento a sustraer de la lista

    Returns:
      new: lista nueva similar a points pero sin el elemento i.
    """
    new = points.copy()
    new.remove(i)

    return new

In [61]:
distancias = df_matrix_dist  #Cambiar si queremos usar tiempos

# Definimos el modelo
modelo_mtz = pywraplp.Solver.CreateSolver('SAT')

list_index_puntos = list(distancias.index)

# Definimos como punto de partida el index 0
partida = list_index_puntos[0]

# uso n como n-1 ya que arranco de indice 0 en la lista. Por esto en mtz no resto 1 ni en u_i
n = max(list_index_puntos)

# Definimos variables

x_ij={i:{j: modelo_mtz.IntVar(0,1,'x_'+str(i)+'_'+str(j)) for j in points_sub(list_index_puntos, i)} for i in list_index_puntos}
u_i = {i: modelo_mtz.IntVar(1,n,'u_'+str(i)) for i in points_sub(list_index_puntos, partida)}

# Definimos la función objetivo
obj_expr = sum(x_ij[i][j] * distancias[i][j] for i in list_index_puntos for j in points_sub(list_index_puntos, i))


#Restricciones:

#1
for j in list_index_puntos:
    modelo_mtz.Add(sum(x_ij[i][j] for i in points_sub(list_index_puntos, j)) == 1)

#2
for i in list_index_puntos:
    modelo_mtz.Add(sum(x_ij[i][j] for j in points_sub(list_index_puntos, i)) == 1)

#3
for i in points_sub(list_index_puntos, partida):
    for j in points_sub(points_sub(list_index_puntos, partida), i):
        modelo_mtz.Add(u_i[i] - u_i[j] + 1 <= n * (1- x_ij[i][j]))


# Solver
modelo_mtz.Minimize(obj_expr)

In [62]:
inicio = time.time()
status = modelo_mtz.Solve()
fin = time.time()
print(f"Tiempo de resolución: {fin - inicio} segundos")

Tiempo de resolución: 0.21219205856323242 segundos


In [63]:
modelo_mtz.Objective().Value()

13370.666604472326

Visualización

In [69]:
def reconstruir_tour(x_ij, partida):
    """
    Reconstruye el tour óptimo a partir de las variables x_ij del modelo MTZ.
    Args:
        x_ij: diccionario de variables de decisión x_ij[i][j]
        partida: nodo de partida (debe ser el mismo que se usó en el modelo)
    Returns:
        tour: lista ordenada de nodos visitados en el tour óptimo
    """
    tour = [partida]
    actual = partida
    visitados = set([partida])
    n = len(x_ij)
    while len(tour) < n:
        for siguiente in x_ij[actual]:
            if x_ij[actual][siguiente].solution_value() > 0.5 and siguiente not in visitados:
                tour.append(siguiente)
                visitados.add(siguiente)
                actual = siguiente
                break
        else:
            # Si no encuentra siguiente, termina (por si hay algún problema)
            break
    if len(tour) > 1:
        tour.append(partida)
    return tour

# Uso:
tour = reconstruir_tour(x_ij, partida)

In [70]:
tour

[257491698,
 257490685,
 257490752,
 257490911,
 1650474841,
 1962759197,
 3364478903,
 257491537,
 1310764161,
 282227009,
 282227014,
 257490605,
 257489978,
 257490364,
 257489020,
 257490219,
 257490114,
 257490112,
 257489792,
 257488992,
 257491698]

In [ ]:
# Visualización del recorrido óptimo usando rutas reales de calles

if len(tour) > 1:
    for i in range(len(tour) - 1):
        origen = tour[i]
        destino = tour[i + 1]
        try:
            # Obtener el camino real en la red vial
            path = nx.shortest_path(G, origen, destino, weight='length')
            # Extraer coordenadas de cada nodo del camino
            coords = [[G.nodes[n]['y'], G.nodes[n]['x']] for n in path]
            folium.PolyLine(locations=coords, color='green', weight=3, opacity=0.7).add_to(m)
        except Exception as e:
            print(f"No se pudo trazar de {origen} a {destino}: {e}")

show_folium_safe(m)

In [72]:
if len(tour) > 1:
    print("Orden de los nodos en el tour óptimo (índices de df_vrp):")
    for idx, node in enumerate(tour):
        # Buscar el índice correspondiente en df_vrp
        df_idx = df_vrp.index[df_vrp['osmid'] == node][0]
        print(f"{idx+1}: índice {df_idx} (osmid {node})")
else:
    print("No se pudo reconstruir el tour óptimo.")

Orden de los nodos en el tour óptimo (índices de df_vrp):
1: índice 0 (osmid 257491698)
2: índice 9 (osmid 257490685)
3: índice 14 (osmid 257490752)
4: índice 8 (osmid 257490911)
5: índice 12 (osmid 1650474841)
6: índice 18 (osmid 1962759197)
7: índice 3 (osmid 3364478903)
8: índice 5 (osmid 257491537)
9: índice 15 (osmid 1310764161)
10: índice 17 (osmid 282227009)
11: índice 10 (osmid 282227014)
12: índice 7 (osmid 257490605)
13: índice 11 (osmid 257489978)
14: índice 19 (osmid 257490364)
15: índice 2 (osmid 257489020)
16: índice 6 (osmid 257490219)
17: índice 4 (osmid 257490114)
18: índice 13 (osmid 257490112)
19: índice 1 (osmid 257489792)
20: índice 16 (osmid 257488992)
21: índice 0 (osmid 257491698)
